<a href="https://colab.research.google.com/github/akatari2024-debug/plane2planeTransformation/blob/main/Copy_of_cam_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q opencv-python-headless gradio matplotlib numpy pytransform3d


In [ ]:
import os, glob, shutil, json, cv2, numpy as np
from pathlib import Path

# Folders
IMDIR = "/content/images"
OUTDIR = "/content/out"
os.makedirs(IMDIR, exist_ok=True)
os.makedirs(OUTDIR, exist_ok=True)

# Pattern constants for your printed board
PATTERN_COLS = 9   # inner corners horizontally
PATTERN_ROWS = 6   # inner corners vertically
SQUARE_SIZE_M = 0.025  # change if your printed squares are not 25 mm
PATTERN_SIZE = (PATTERN_COLS, PATTERN_ROWS)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

class IO:
    @staticmethod
    def list_images(folder=IMDIR):
        exts = (".jpg", ".jpeg", ".JPG", ".JPEG")
        return sorted([str(p) for p in Path(folder).glob("*") if p.suffix in exts])

    @staticmethod
    def save_json(data, path="calibration.json"):
        with open(path, "w") as f:
            json.dump(data, f, indent=2)

    @staticmethod
    def load_json(path="calibration.json"):
        with open(path, "r") as f:
            return json.load(f)

class Board:
    @staticmethod
    def object_points(pattern_size=PATTERN_SIZE, square_size=SQUARE_SIZE_M):
        cols, rows = pattern_size
        objp = np.zeros((cols*rows, 3), np.float32)
        objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
        objp *= float(square_size)
        return objp

class Calib:
    @staticmethod
    def find_corners(image_paths, pattern_size=PATTERN_SIZE, subpix=True):
        objp = Board.object_points(pattern_size)
        objpoints, imgpoints, corners_list, ok_mask = [], [], [], []
        img_shape = None

        for p in image_paths:
            img = cv2.imread(p)
            if img is None:
                corners_list.append(None); ok_mask.append(False); continue
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_shape = gray.shape[::-1]

            ret, corners = cv2.findChessboardCorners(
                gray, pattern_size,
                flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            )
            if ret and subpix:
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)
                corners = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

            if ret:
                objpoints.append(objp)
                imgpoints.append(corners)
                corners_list.append(corners)
                ok_mask.append(True)
            else:
                corners_list.append(None)
                ok_mask.append(False)

        return objpoints, imgpoints, img_shape, corners_list, ok_mask

    @staticmethod
    def run(image_paths, pattern_size=PATTERN_SIZE, square_size=SQUARE_SIZE_M):
        objpoints, imgpoints, img_shape, corners_list, ok_mask = Calib.find_corners(
            image_paths, pattern_size
        )
        if not objpoints:
            raise RuntimeError("No chessboard detected. Check pattern size (9×6) and image quality.")

        ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
            objpoints, imgpoints, img_shape, None, None
        )

        # Per-image reprojection errors (nice for your report)
        per_image_err = []
        K_np, D_np = np.array(K), np.array(dist)
        for i in range(len(objpoints)):
            pts2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K_np, D_np)
            err = cv2.norm(imgpoints[i], pts2, cv2.NORM_L2) / len(pts2)
            per_image_err.append(float(err))

        results = {
            "rms_reprojection_error": float(ret),
            "per_image_reprojection_error": per_image_err,
            "K": K.tolist(),
            "dist": dist.tolist(),
            "rvecs": [r.flatten().tolist() for r in rvecs],
            "tvecs": [t.flatten().tolist() for t in tvecs],
            "width": int(img_shape[0]),
            "height": int(img_shape[1]),
            "pattern_size": list(pattern_size),
            "square_size_m": float(square_size),
            "used_images": [image_paths[i] for i, ok in enumerate(ok_mask) if ok]
        }
        return results, corners_list, ok_mask

class Overlay:
    @staticmethod
    def draw_corners(img, corners, pattern_size=PATTERN_SIZE):
        vis = img.copy()
        cv2.drawChessboardCorners(vis, pattern_size, corners, True)
        return vis

    @staticmethod
    def draw_axes(img, K, dist, rvec, tvec, axis_len=0.1):
        vis = img.copy()
        axis = np.float32([[0,0,0],[axis_len,0,0],[0,axis_len,0],[0,0,axis_len]])
        pts, _ = cv2.projectPoints(axis, rvec, tvec, K, dist)
        pts = pts.reshape(-1,2).astype(int)
        cv2.line(vis, tuple(pts[0]), tuple(pts[1]), (0,0,255), 3)   # x red
        cv2.line(vis, tuple(pts[0]), tuple(pts[2]), (0,255,0), 3)   # y green
        cv2.line(vis, tuple(pts[0]), tuple(pts[3]), (255,0,0), 3)   # z blue
        return vis

    @staticmethod
    def undistort(img, K, dist):
        h, w = img.shape[:2]
        newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), 1, (w,h))
        return cv2.undistort(img, K, dist, None, newK)

class Viz:
    @staticmethod
    def camera_centers_board_frame(rvecs, tvecs):
        Cs = []
        for rvec, tvec in zip(rvecs, tvecs):
            R, _ = cv2.Rodrigues(np.array(rvec).reshape(3,1))
            t = np.array(tvec).reshape(3,1)
            C = -R.T @ t  # camera center in board frame
            Cs.append(C.flatten())
        return np.array(Cs)

    @staticmethod
    def plot_camera_poses(rvecs, tvecs, square_size=SQUARE_SIZE_M, grid_size=(10,7), save_path=f"{OUTDIR}/camera_poses.png"):
        fig = plt.figure(figsize=(6,6))
        ax = fig.add_subplot(111, projection="3d")

        gx, gy = grid_size
        xs = np.arange(0, gx+1) * square_size
        ys = np.arange(0, gy+1) * square_size
        for x in xs: ax.plot([x,x],[0,gy*square_size],[0,0], linewidth=1)
        for y in ys: ax.plot([0,gx*square_size],[y,y],[0,0], linewidth=1)

        Cs = Viz.camera_centers_board_frame(rvecs, tvecs)
        if len(Cs): ax.scatter(Cs[:,0], Cs[:,1], Cs[:,2], s=30, marker='o')

        ax.set_xlabel('X (m)'); ax.set_ylabel('Y (m)'); ax.set_zlabel('Z (m)')
        ax.view_init(elev=25, azim=-60)
        ax.set_title("Estimated Camera Poses (board frame)")
        fig.tight_layout()
        fig.savefig(save_path, dpi=160)
        plt.close(fig)
        return save_path

    @staticmethod
    def sample_overlays(image_paths, corners_list, rvecs, tvecs, K, D, pattern_size=PATTERN_SIZE, max_n=8):
        outs, used = [], 0
        for p, corners, rvec, tvec in zip(image_paths, corners_list, rvecs, tvecs):
            if corners is None: continue
            img = cv2.imread(p)
            img = Overlay.draw_corners(img, corners, pattern_size)
            img = Overlay.draw_axes(img, K, D, np.array(rvec), np.array(tvec), axis_len=0.1)
            outp = f"{OUTDIR}/overlay_{Path(p).stem}.jpg"
            cv2.imwrite(outp, img)
            outs.append(outp)
            used += 1
            if used >= max_n: break
        return outs

    @staticmethod
    def undistort_side_by_side(img_path, K, D, save_path=f"{OUTDIR}/undistort_preview.jpg"):
        img = cv2.imread(img_path)
        und = Overlay.undistort(img, K, D)
        side = np.hstack([img, und])
        cv2.imwrite(save_path, side)
        return save_path


In [ ]:
import gradio as gr

STATE = {"results": None, "corners_list": None, "ok_mask": None, "image_paths": None}

def ui_reset_images():
    if os.path.exists(IMDIR):
        for f in glob.glob(f"{IMDIR}/*"):
            try: os.remove(f)
            except: pass
    os.makedirs(IMDIR, exist_ok=True)
    return "Image folder reset."

def ui_upload(files):
    os.makedirs(IMDIR, exist_ok=True)
    for f in files: shutil.copy(f.name, IMDIR)
    imgs = IO.list_images(IMDIR)
    return f"Uploaded {len(files)} files. JPEGs present: {len(imgs)}."

def ui_calibrate(square_meters):
    image_paths = IO.list_images(IMDIR)
    if len(image_paths) < 5:
        return "Need at least 5 images (ideally 15–20)."

    # run calibration (pattern 9x6 locked for your board)
    global SQUARE_SIZE_M
    SQUARE_SIZE_M = float(square_meters)

    results, corners_list, ok_mask = Calib.run(
        image_paths, pattern_size=PATTERN_SIZE, square_size=SQUARE_SIZE_M
    )
    IO.save_json(results, "calibration.json")

    STATE["results"] = results
    STATE["corners_list"] = corners_list
    STATE["ok_mask"] = ok_mask
    STATE["image_paths"] = [p for p, ok in zip(image_paths, ok_mask) if ok]

    msg = [
        "Calibration complete ✅",
        f"RMS reprojection error: {results['rms_reprojection_error']:.4f} px",
        f"Used images: {len(results['used_images'])}/{len(image_paths)}",
        f"Saved → calibration.json",
    ]
    return "\n".join(msg)

def ui_show_poses():
    r = STATE["results"]
    if not r: return None, "Run calibration first."
    path = Viz.plot_camera_poses(r["rvecs"], r["tvecs"], square_size=r["square_size_m"])
    return path, "Camera poses plotted."

def ui_show_overlays(max_n):
    r = STATE["results"]
    if not r: return [], "Run calibration first."
    K = np.array(r["K"]); D = np.array(r["dist"])
    outs = Viz.sample_overlays(STATE["image_paths"], STATE["corners_list"], r["rvecs"], r["tvecs"], K, D, pattern_size=PATTERN_SIZE, max_n=int(max_n))
    return outs, f"Generated {len(outs)} overlay images."

def ui_undistort_preview(image_name):
    r = STATE["results"]
    if not r: return None
    K = np.array(r["K"]); D = np.array(r["dist"])
    path = Viz.undistort_side_by_side(f"{IMDIR}/{image_name}", K, D)
    return path

def ui_list_image_names():
    return [Path(p).name for p in IO.list_images(IMDIR)]

with gr.Blocks(title="Camera Calibration (OpenCV + Gradio)", analytics_enabled=False) as demo:
    gr.Markdown("## Camera Calibration (OpenCV + Gradio) — 9×6 chessboard")

    with gr.Tab("1) Upload"):
        gr.Markdown("Upload **.jpg/.jpeg** images. They will be saved to `/content/images/`.")
        upload = gr.File(label="Upload calibration images", file_count="multiple", file_types=[".jpg", ".jpeg"])
        up_btn = gr.Button("Save to /content/images")
        reset_btn = gr.Button("Reset /content/images")
        up_out = gr.Textbox(label="Status")
        up_btn.click(ui_upload, inputs=upload, outputs=up_out)
        reset_btn.click(ui_reset_images, outputs=up_out)

    with gr.Tab("2) Calibrate"):
        gr.Markdown("Pattern size is fixed to your board: **9 × 6 inner corners**.")
        cols = gr.Number(value=PATTERN_COLS, precision=0, label="Pattern columns", interactive=False)
        rows = gr.Number(value=PATTERN_ROWS, precision=0, label="Pattern rows", interactive=False)
        sq   = gr.Number(value=SQUARE_SIZE_M, label="Square size (meters)")
        run  = gr.Button("Run Calibration")
        cal_out = gr.Textbox(label="Calibration log", lines=6)
        run.click(ui_calibrate, inputs=[sq], outputs=cal_out)

        gr.Markdown("Download results:")
        cal_file = gr.File(label="calibration.json")
        def _serve_json(): return "calibration.json"
        run.click(_serve_json, outputs=cal_file)

    with gr.Tab("3) Visualize"):
        poses_btn = gr.Button("Plot Camera Poses")
        poses_img = gr.Image(label="Camera poses (board frame)")
        poses_log = gr.Textbox(label="Log")
        poses_btn.click(ui_show_poses, outputs=[poses_img, poses_log])

        gr.Markdown("### Sample Overlays (corners + world axes)")
        maxn = gr.Slider(1, 10, value=8, step=1, label="How many images to render?")
        overlay_btn = gr.Button("Generate overlays")
        overlay_gallery = gr.Gallery(label="Overlays", height=300)
        overlay_log = gr.Textbox(label="Log")
        overlay_btn.click(ui_show_overlays, inputs=maxn, outputs=[overlay_gallery, overlay_log])

        gr.Markdown("### Undistortion preview (before | after)")
        refresh_btn = gr.Button("Refresh image list")
        name_dd = gr.Dropdown(choices=[], label="Pick an uploaded image")
        prev_btn = gr.Button("Show undistortion preview")
        prev_img = gr.Image(label="Undistort preview")
        refresh_btn.click(ui_list_image_names, outputs=name_dd)
        prev_btn.click(ui_undistort_preview, inputs=name_dd, outputs=prev_img)

demo.launch(inline=True, share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

next step to do
1. print chess board
2. take 15-20 photo
3. upload and run calibaration using gradio ai

In [ ]:
import glob
overlays = glob.glob("/content/out/overlay_*.jpg")
print("Found overlays:", len(overlays))
print(overlays[:5])  # show first 5 filenames


Found overlays: 8
['/content/out/overlay_WhatsApp Image 2025-09-02 at 5.54.29 PM 4.jpg', '/content/out/overlay_WhatsApp Image 2025-09-02 at 5.54.29 PM 11.jpg', '/content/out/overlay_WhatsApp Image 2025-09-02 at 5.54.29 PM 10.jpg', '/content/out/overlay_WhatsApp Image 2025-09-02 at 5.54.29 PM 3.jpg', '/content/out/overlay_WhatsApp Image 2025-09-02 at 5.54.29 PM 13.jpg']


In [ ]:
import cv2, matplotlib.pyplot as plt

if overlays:
    img = cv2.imread(overlays[0])
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Overlay preview")
    plt.axis("off")
    plt.show()


In [ ]:
from google.colab import files
for f in overlays:
    files.download(f)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2

def ui_show_overlays(max_n):
    r = STATE["results"]
    if not r: return [], "Run calibration first."
    K = np.array(r["K"]); D = np.array(r["dist"])
    outs = Viz.sample_overlays(STATE["image_paths"], STATE["corners_list"], r["rvecs"], r["tvecs"], K, D, pattern_size=PATTERN_SIZE, max_n=int(max_n))

    # Load images instead of paths
    imgs = [cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB) for p in outs]
    return imgs, f"Generated {len(imgs)} overlay images."


In [ ]:
# Count images and preview first 3 shapes
files = IO.list_images(IMDIR)
print("Found images:", len(files))
for f in files[:3]:
    im = cv2.imread(f)
    print(Path(f).name, "→", None if im is None else im.shape)

# Quick corner check on a few images
for f in files[:5]:
    img = cv2.imread(f)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, PATTERN_SIZE, None)
    print(Path(f).name, "corners:", "FOUND" if ret else "FAILED")
